#Homework 2 - Spark

## Lettura dei dati

In [0]:
import requests
from pyspark.sql.functions import isnan, when, count, isnull

In [0]:
# local_path_popolazione_regioni = "dbfs:/FileStore/shared_uploads/gu.dichiara@studenti.unina.it/popolazione_regioni.csv"
# local_path_dati_covid_province = "dbfs:/FileStore/shared_uploads/gu.dichiara@studenti.unina.it/dati_covid_province.csv"
# local_path_dati_covid_regioni = "dbfs:/FileStore/shared_uploads/gu.dichiara@studenti.unina.it/dati_covid_regioni.csv"
# local_path_dati_vaccini_somministrazioni = "dbfs:/FileStore/shared_uploads/gu.dichiara@studenti.unina.it/dati_vaccini_somministrazioni.csv"
# local_path_dati_vaccini_consegne = "dbfs:/FileStore/shared_uploads/gu.dichiara@studenti.unina.it/dati_vaccini_consegne.csv"

local_path_popolazione_regioni = "dbfs:/FileStore/shared_uploads/fabio.dandrea@studenti.unina.it/popolazione_regioni.csv"
local_path_dati_covid_province = "dbfs:/FileStore/shared_uploads/fabio.dandrea@studenti.unina.it/dati_covid_province.csv"
local_path_dati_covid_regioni = "dbfs:/FileStore/shared_uploads/fabio.dandrea@studenti.unina.it/dati_covid_regioni.csv"
local_path_dati_vaccini_somministrazioni = "dbfs:/FileStore/shared_uploads/fabio.dandrea@studenti.unina.it/dati_vaccini_somministrazioni.csv"
local_path_dati_vaccini_consegne = "dbfs:/FileStore/shared_uploads/fabio.dandrea@studenti.unina.it/dati_vaccini_consegne.csv"

popolazione_regioni_URL = "https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-statistici-riferimento/popolazione-istat-regione-range.csv"
dati_covid_province_URL = "https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-province/dpc-covid19-ita-province.csv"
dati_covid_regioni_URL = "https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv"
dati_vaccini_somministrazioni_URL = "https://raw.githubusercontent.com/italia/covid19-opendata-vaccini/master/dati/somministrazioni-vaccini-latest.csv"
dati_vaccini_consegne_URL = "https://raw.githubusercontent.com/italia/covid19-opendata-vaccini/master/dati/consegne-vaccini-latest.csv"

dbutils.fs.put(local_path_popolazione_regioni, requests.get(popolazione_regioni_URL).text, True)
dbutils.fs.put(local_path_dati_covid_province, requests.get(dati_covid_province_URL).text, True)
dbutils.fs.put(local_path_dati_covid_regioni, requests.get(dati_covid_regioni_URL).text, True)
dbutils.fs.put(local_path_dati_vaccini_somministrazioni, requests.get(dati_vaccini_somministrazioni_URL).text, True)
dbutils.fs.put(local_path_dati_vaccini_consegne, requests.get(dati_vaccini_consegne_URL).text, True)

popolazione_regioni = spark.read.option("inferSchema", "true").option("header", "true").csv(local_path_popolazione_regioni)
dati_covid_province = spark.read.option("inferSchema", "true").option("header", "true").csv(local_path_dati_covid_province)
dati_covid_regioni = spark.read.option("inferSchema", "true").option("header", "true").csv(local_path_dati_covid_regioni)
dati_vaccini_somministrazioni = spark.read.option("inferSchema", "true").option("header", "true").csv(local_path_dati_vaccini_somministrazioni)
dati_vaccini_consegne = spark.read.option("inferSchema", "true").option("header", "true").csv(local_path_dati_vaccini_consegne)

Wrote 17300 bytes.
Wrote 5414859 bytes.
Wrote 1481207 bytes.
Wrote 3759833 bytes.
Wrote 54925 bytes.

## Preprocessing

### Popolazione Regioni
Si eliminano le colonne che non sono di interesse.

In [0]:
popolazione_regioni = popolazione_regioni.drop('codice_regione', 'latitudine_regione', 'longitudine_regione')
popolazione_regioni.show()

# popolazione_regioni.groupBy('denominazione_regione').sum('totale_generale').orderBy('sum(totale_generale)', ascending=False).show()

+-------------+------------------+-------------+---------------------+-------------+---------+----------------------+-----------------------+---------------+
codice_nuts_1|descrizione_nuts_1|codice_nuts_2|denominazione_regione|sigla_regione|range_eta|totale_genere_maschile|totale_genere_femminile|totale_generale|
+-------------+------------------+-------------+---------------------+-------------+---------+----------------------+-----------------------+---------------+
 ITF| Sud| ITF1| Abruzzo| ABR| 0-15| 87423| 82474| 169897|
 ITF| Sud| ITF1| Abruzzo| ABR| 16-19| 24402| 22349| 46751|
 ITF| Sud| ITF1| Abruzzo| ABR| 20-29| 67472| 62611| 130083|
 ITF| Sud| ITF1| Abruzzo| ABR| 30-39| 76472| 74241| 150713|
 ITF| Sud| ITF1| Abruzzo| ABR| 40-49| 94745| 95315| 190060|
 ITF| Sud| ITF1| Abruzzo| ABR| 50-59| 100048| 104109| 204157|
 ITF| Sud| ITF1| Abruzzo| ABR| 60-69| 80545| 87160| 167705|
 ITF| Sud| ITF1| Abruzzo| ABR| 70-79| 60810| 69762| 130572|
 ITF| Sud| ITF1| Abruzzo| ABR| 80-89| 34181| 50307| 84488|
 ITF| Sud| ITF1| Abruzzo| ABR| 90+| 5645| 13870| 19515|
 ITF| Sud| ITF5| Basilicata| BAS| 0-15| 36113| 33881| 69994|
 ITF| Sud| ITF5| Basilicata| BAS| 16-19| 11598| 10434| 22032|
 ITF| Sud| ITF5| Basilicata| BAS| 20-29| 32068| 29115| 61183|
 ITF| Sud| ITF5| Basilicata| BAS| 30-39| 33556| 31648| 65204|
 ITF| Sud| ITF5| Basilicata| BAS| 40-49| 39260| 39207| 78467|
 ITF| Sud| ITF5| Basilicata| BAS| 50-59| 43150| 44672| 87822|
 ITF| Sud| ITF5| Basilicata| BAS| 60-69| 35360| 37457| 72817|
 ITF| Sud| ITF5| Basilicata| BAS| 70-79| 24050| 27755| 51805|
 ITF| Sud| ITF5| Basilicata| BAS| 80-89| 14507| 21600| 36107|
 ITF| Sud| ITF5| Basilicata| BAS| 90+| 2488| 5335| 7823|
+-------------+------------------+-------------+---------------------+-------------+---------+----------------------+-----------------------+---------------+
only showing top 20 rows

Si verifica la presenza di *missing values*.

In [0]:
popolazione_regioni.select([count(when(isnull(c), c)).alias(c) for c in popolazione_regioni.columns]).show()

+-------------+------------------+-------------+---------------------+-------------+---------+----------------------+-----------------------+---------------+
codice_nuts_1|descrizione_nuts_1|codice_nuts_2|denominazione_regione|sigla_regione|range_eta|totale_genere_maschile|totale_genere_femminile|totale_generale|
+-------------+------------------+-------------+---------------------+-------------+---------+----------------------+-----------------------+---------------+
 0| 0| 0| 0| 0| 0| 0| 0| 0|
+-------------+------------------+-------------+---------------------+-------------+---------+----------------------+-----------------------+---------------+

Si verifica il numero di regioni e la loro denominazione.

In [0]:
num_regioni = popolazione_regioni.groupBy('denominazione_regione') \
                .count() \
                .count()
print('Il numero di regioni è:', num_regioni)
print('')
popolazione_regioni.groupBy('descrizione_nuts_1', 'denominazione_regione', 'sigla_regione') \
                    .count() \
                    .drop('count') \
                    .orderBy('descrizione_nuts_1').show(num_regioni)

Il numero di regioni è: 21

+------------------+---------------------+-------------+
descrizione_nuts_1|denominazione_regione|sigla_regione|
+------------------+---------------------+-------------+
 Centro| Umbria| UMB|
 Centro| Toscana| TOS|
 Centro| Marche| MAR|
 Centro| Lazio| LAZ|
 Isole| Sardegna| SAR|
 Isole| Sicilia| SIC|
 Nord-Est| Bolzano| BOL|
 Nord-Est| Emilia-Romagna| EMR|
 Nord-Est| Trento| TRE|
 Nord-Est| Veneto| VEN|
 Nord-Est| Friuli Venezia Gi...| FVG|
 Nord-Ovest| Lombardia| LOM|
 Nord-Ovest| Liguria| LIG|
 Nord-Ovest| Valle d'Aosta| VDA|
 Nord-Ovest| Piemonte| PIE|
 Sud| Campania| CAM|
 Sud| Abruzzo| ABR|
 Sud| Calabria| CAL|
 Sud| Molise| MOL|
 Sud| Puglia| PUG|
 Sud| Basilicata| BAS|
+------------------+---------------------+-------------+

### Dati Covid Province
Si eliminano le colonne che non sono di interesse.

In [0]:
dati_covid_province = dati_covid_province.dropna(subset='sigla_provincia') \
                        .drop('stato', 'codice_regione', 'codice_provincia', 'lat', 'long', 'note')
dati_covid_province.show()

+-------------------+---------------------+-----------------------+---------------+-----------+-------------+-------------+-------------+
 data|denominazione_regione|denominazione_provincia|sigla_provincia|totale_casi|codice_nuts_1|codice_nuts_2|codice_nuts_3|
+-------------------+---------------------+-----------------------+---------------+-----------+-------------+-------------+-------------+
2020-02-24 18:00:00| Abruzzo| L'Aquila| AQ| 0| null| null| null|
2020-02-24 18:00:00| Abruzzo| Teramo| TE| 0| null| null| null|
2020-02-24 18:00:00| Abruzzo| Pescara| PE| 0| null| null| null|
2020-02-24 18:00:00| Abruzzo| Chieti| CH| 0| null| null| null|
2020-02-24 18:00:00| Basilicata| Potenza| PZ| 0| null| null| null|
2020-02-24 18:00:00| Basilicata| Matera| MT| 0| null| null| null|
2020-02-24 18:00:00| Calabria| Cosenza| CS| 0| null| null| null|
2020-02-24 18:00:00| Calabria| Catanzaro| CZ| 0| null| null| null|
2020-02-24 18:00:00| Calabria| Reggio di Calabria| RC| 0| null| null| null|
2020-02-24 18:00:00| Calabria| Crotone| KR| 0| null| null| null|
2020-02-24 18:00:00| Calabria| Vibo Valentia| VV| 0| null| null| null|
2020-02-24 18:00:00| Campania| Caserta| CE| 0| null| null| null|
2020-02-24 18:00:00| Campania| Benevento| BN| 0| null| null| null|
2020-02-24 18:00:00| Campania| Napoli| NA| 0| null| null| null|
2020-02-24 18:00:00| Campania| Avellino| AV| 0| null| null| null|
2020-02-24 18:00:00| Campania| Salerno| SA| 0| null| null| null|
2020-02-24 18:00:00| Emilia-Romagna| Piacenza| PC| 0| null| null| null|
2020-02-24 18:00:00| Emilia-Romagna| Parma| PR| 0| null| null| null|
2020-02-24 18:00:00| Emilia-Romagna| Reggio nell'Emilia| RE| 0| null| null| null|
2020-02-24 18:00:00| Emilia-Romagna| Modena| MO| 0| null| null| null|
+-------------------+---------------------+-----------------------+---------------+-----------+-------------+-------------+-------------+
only showing top 20 rows

Si verifica la presenza di *missing values*.

In [0]:
dati_covid_province.select([count(when(isnull(c), c)).alias(c) for c in dati_covid_province.drop('data').columns]).show()

+---------------------+-----------------------+---------------+-----------+-------------+-------------+-------------+
denominazione_regione|denominazione_provincia|sigla_provincia|totale_casi|codice_nuts_1|codice_nuts_2|codice_nuts_3|
+---------------------+-----------------------+---------------+-----------+-------------+-------------+-------------+
 0| 0| 0| 0| 34882| 34882| 34882|
+---------------------+-----------------------+---------------+-----------+-------------+-------------+-------------+

Si recuperano i valori mancanti dei codici nuts dalla tabella stessa.

In [0]:
data = dati_covid_province.groupBy('codice_nuts_1', 'codice_nuts_2', 'codice_nuts_3', 'denominazione_regione', 'denominazione_provincia') \
                          .count() \
                          .drop('count') \
                          .dropna() \
                          .distinct()

dati_covid_province = dati_covid_province.withColumnRenamed('codice_nuts_1', 'codice_nuts_1_old')
dati_covid_province = dati_covid_province.withColumnRenamed('codice_nuts_2', 'codice_nuts_2_old')
dati_covid_province = dati_covid_province.withColumnRenamed('codice_nuts_3', 'codice_nuts_3_old')

dati_covid_province = dati_covid_province.join(data, on=['denominazione_regione', 'denominazione_provincia'], how='left')
dati_covid_province = dati_covid_province.drop('codice_nuts_1_old', 'codice_nuts_2_old', 'codice_nuts_3_old')

In [0]:
dati_covid_province.select([count(when(isnull(c), c)).alias(c) for c in dati_covid_province.drop('data').columns]).show()

+---------------------+-----------------------+---------------+-----------+-------------+-------------+-------------+
denominazione_regione|denominazione_provincia|sigla_provincia|totale_casi|codice_nuts_1|codice_nuts_2|codice_nuts_3|
+---------------------+-----------------------+---------------+-----------+-------------+-------------+-------------+
 0| 0| 0| 0| 0| 0| 0|
+---------------------+-----------------------+---------------+-----------+-------------+-------------+-------------+

Si verifica il numero di regioni e la loro denominazione.

In [0]:
num_regioni = dati_covid_province.groupBy('denominazione_regione') \
                                    .count() \
                                    .count()
print('Il numero di regioni è:', num_regioni)
print()
dati_covid_province.groupBy('denominazione_regione') \
                    .count() \
                    .drop('count') \
                    .show(num_regioni)

Il numero di regioni è: 21

+---------------------+
denominazione_regione|
+---------------------+
 Emilia-Romagna|
 Liguria|
 Lazio|
 Sicilia|
 Toscana|
 Abruzzo|
 Piemonte|
 Calabria|
 P.A. Trento|
 Veneto|
 Lombardia|
 Puglia|
 Sardegna|
 Umbria|
 P.A. Bolzano|
 Molise|
 Friuli Venezia Gi...|
 Valle d'Aosta|
 Marche|
 Basilicata|
 Campania|
+---------------------+

`dati_covid_province.denominazione_regione` non coincide con `popolazione_regioni.denominazione_regione`.

In [0]:
join1 = popolazione_regioni.join(dati_covid_province, on='denominazione_regione', how='anti') \
                    .groupBy('codice_nuts_2', 'denominazione_regione') \
                    .count() \
                    .drop('count') \
                    .withColumnRenamed('denominazione_regione', 'popolazione_regioni_denominazione_regione')

join2 = dati_covid_province.join(popolazione_regioni, on='denominazione_regione', how='anti') \
                    .groupBy('codice_nuts_2', 'denominazione_regione') \
                    .count() \
                    .drop('count') \
                    .withColumnRenamed('denominazione_regione', 'dati_covid_province_denominazione_regione')

join1.join(join2, on='codice_nuts_2').show()

+-------------+-----------------------------------------+-----------------------------------------+
codice_nuts_2|popolazione_regioni_denominazione_regione|dati_covid_province_denominazione_regione|
+-------------+-----------------------------------------+-----------------------------------------+
 ITH1| Bolzano| P.A. Bolzano|
 ITH2| Trento| P.A. Trento|
+-------------+-----------------------------------------+-----------------------------------------+

In [0]:
dati_covid_province = dati_covid_province.replace('P.A. Trento', value='Trento', subset='denominazione_regione')
dati_covid_province = dati_covid_province.replace('P.A. Bolzano', value='Bolzano', subset='denominazione_regione')

Si verifica il numero di province e la loro denominazione.

In [0]:
num_province = dati_covid_province.groupBy('denominazione_provincia') \
                                    .count() \
                                    .count()
print('Il numero di province è:', num_province)
print()
dati_covid_province.groupBy('denominazione_provincia', 'sigla_provincia') \
                      .count() \
                      .drop('count') \
                      .orderBy('denominazione_provincia') \
                      .show(num_province)

Il numero di province è: 107

+-----------------------+---------------+
denominazione_provincia|sigla_provincia|
+-----------------------+---------------+
 Agrigento| AG|
 Alessandria| AL|
 Ancona| AN|
 Aosta| AO|
 Arezzo| AR|
 Ascoli Piceno| AP|
 Asti| AT|
 Avellino| AV|
 Bari| BA|
 Barletta-Andria-T...| BT|
 Belluno| BL|
 Benevento| BN|
 Bergamo| BG|
 Biella| BI|
 Bologna| BO|
 Bolzano| BZ|
 Brescia| BS|
 Brindisi| BR|
 Cagliari| CA|
 Caltanissetta| CL|
 Campobasso| CB|
 Caserta| CE|
 Catania| CT|
 Catanzaro| CZ|
 Chieti| CH|
 Como| CO|
 Cosenza| CS|
 Cremona| CR|
 Crotone| KR|
 Cuneo| CN|
 Enna| EN|
 Fermo| FM|
 Ferrara| FE|
 Firenze| FI|
 Foggia| FG|
 Forlì-Cesena| FC|
 Frosinone| FR|
 Genova| GE|
 Gorizia| GO|
 Grosseto| GR|
 Imperia| IM|
 Isernia| IS|
 L'Aquila| AQ|
 La Spezia| SP|
 Latina| LT|
 Lecce| LE|
 Lecco| LC|
 Livorno| LI|
 Lodi| LO|
 Lucca| LU|
 Macerata| MC|
 Mantova| MN|
 Massa Carrara| MS|
 Matera| MT|
 Messina| ME|
 Milano| MI|
 Modena| MO|
 Monza e della Bri...| MB|
 Napoli| NA|
 Novara| NO|
 Nuoro| NU|
 Oristano| OR|
 Padova| PD|
 Palermo| PA|
 Parma| PR|
 Pavia| PV|
 Perugia| PG|
 Pesaro e Urbino| PU|
 Pescara| PE|
 Piacenza| PC|
 Pisa| PI|
 Pistoia| PT|
 Pordenone| PN|
 Potenza| PZ|
 Prato| PO|
 Ragusa| RG|
 Ravenna| RA|
 Reggio di Calabria| RC|
 Reggio nell'Emilia| RE|
 Rieti| RI|
 Rimini| RN|
 Roma| RM|
 Rovigo| RO|
 Salerno| SA|
 Sassari| SS|
 Savona| SV|
 Siena| SI|
 Siracusa| SR|
 Sondrio| SO|
 Sud Sardegna| SU|
 Taranto| TA|
 Teramo| TE|
 Terni| TR|
 Torino| TO|
 Trapani| TP|
 Trento| TN|
 Treviso| TV|
 Trieste| TS|
 Udine| UD|
 Varese| VA|
 Venezia| VE|
 Verbano-Cusio-Ossola| VB|
 Vercelli| VC|
 Verona| VR|
 Vibo Valentia| VV|
 Vicenza| VI|
 Viterbo| VT|
+-----------------------+---------------+

### Dati Covid Regioni
Si eliminano le colonne che non sono di interesse.

In [0]:
dati_covid_regioni = dati_covid_regioni.drop('stato', 'lat', 'long', 'note_test', 'note_casi', 'note',
                                             'casi_da_sospetto_diagnostico', 'casi_da_screening',
                                             'casi_testati', 'ingressi_terapia_intensiva', 
                                             'totale_positivi_test_molecolare', 'totale_positivi_test_antigenico_rapido',
                                             'tamponi_test_molecolare', 'tamponi_test_antigenico_rapido')

Si verifica la presenza di *missing values*.

In [0]:
dati_covid_regioni.select([count(when(isnull(c), c)).alias(c) for c in dati_covid_regioni.drop('data').columns]).show()

+--------------+---------------------+----------------------+-----------------+--------------------+----------------------+---------------+--------------------------+--------------+---------------+--------+-----------+-------+-------------+-------------+
codice_regione|denominazione_regione|ricoverati_con_sintomi|terapia_intensiva|totale_ospedalizzati|isolamento_domiciliare|totale_positivi|variazione_totale_positivi|nuovi_positivi|dimessi_guariti|deceduti|totale_casi|tamponi|codice_nuts_1|codice_nuts_2|
+--------------+---------------------+----------------------+-----------------+--------------------+----------------------+---------------+--------------------------+--------------+---------------+--------+-----------+-------+-------------+-------------+
 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 6846| 6847|
+--------------+---------------------+----------------------+-----------------+--------------------+----------------------+---------------+--------------------------+--------------+---------------+--------+-----------+-------+-------------+-------------+

Si recuperano i valori mancanti dei codici nuts dalla tabella stessa.

In [0]:
data = dati_covid_regioni.groupBy('codice_nuts_1', 'codice_nuts_2', 'denominazione_regione') \
                          .count() \
                          .drop('count') \
                          .dropna() \
                          .distinct()

dati_covid_regioni = dati_covid_regioni.withColumnRenamed('codice_nuts_1', 'codice_nuts_1_old')
dati_covid_regioni = dati_covid_regioni.withColumnRenamed('codice_nuts_2', 'codice_nuts_2_old')

dati_covid_regioni = dati_covid_regioni.join(data, on=['denominazione_regione'], how='left')
dati_covid_regioni = dati_covid_regioni.drop('codice_nuts_1_old', 'codice_nuts_2_old')

In [0]:
dati_covid_regioni.select([count(when(isnull(c), c)).alias(c) for c in dati_covid_regioni.drop('data').columns]).show()

+---------------------+--------------+----------------------+-----------------+--------------------+----------------------+---------------+--------------------------+--------------+---------------+--------+-----------+-------+-------------+-------------+
denominazione_regione|codice_regione|ricoverati_con_sintomi|terapia_intensiva|totale_ospedalizzati|isolamento_domiciliare|totale_positivi|variazione_totale_positivi|nuovi_positivi|dimessi_guariti|deceduti|totale_casi|tamponi|codice_nuts_1|codice_nuts_2|
+---------------------+--------------+----------------------+-----------------+--------------------+----------------------+---------------+--------------------------+--------------+---------------+--------+-----------+-------+-------------+-------------+
 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
+---------------------+--------------+----------------------+-----------------+--------------------+----------------------+---------------+--------------------------+--------------+---------------+--------+-----------+-------+-------------+-------------+

Si verifica il numero di regioni e la loro denominazione.

In [0]:
num_regioni = dati_covid_regioni.groupBy('denominazione_regione') \
                                    .count() \
                                    .count()
print('Il numero di regioni è:', num_regioni)
print()
dati_covid_regioni.groupBy('denominazione_regione') \
                    .count() \
                    .drop('count') \
                    .show(num_regioni)

Il numero di regioni è: 21

+---------------------+
denominazione_regione|
+---------------------+
 Emilia-Romagna|
 Liguria|
 Lazio|
 Sicilia|
 Toscana|
 Abruzzo|
 Piemonte|
 Calabria|
 P.A. Trento|
 Veneto|
 Lombardia|
 Puglia|
 Sardegna|
 Umbria|
 P.A. Bolzano|
 Molise|
 Friuli Venezia Gi...|
 Valle d'Aosta|
 Marche|
 Basilicata|
 Campania|
+---------------------+

`dati_covid_regioni.denominazione_regione` non coincide con `popolazione_regioni.denominazione_regione`.

In [0]:
join1 = popolazione_regioni.join(dati_covid_regioni, on='denominazione_regione', how='anti') \
                    .groupBy('codice_nuts_2', 'denominazione_regione') \
                    .count() \
                    .drop('count') \
                    .withColumnRenamed('denominazione_regione', 'popolazione_regioni_denominazione_regione')

join2 = dati_covid_regioni.join(popolazione_regioni, on='denominazione_regione', how='anti') \
                    .groupBy('codice_nuts_2', 'denominazione_regione') \
                    .count() \
                    .drop('count') \
                    .withColumnRenamed('denominazione_regione', 'dati_covid_regioni_denominazione_regione')

join1.join(join2, on='codice_nuts_2').show()

+-------------+-----------------------------------------+----------------------------------------+
codice_nuts_2|popolazione_regioni_denominazione_regione|dati_covid_regioni_denominazione_regione|
+-------------+-----------------------------------------+----------------------------------------+
 ITH1| Bolzano| P.A. Bolzano|
 ITH2| Trento| P.A. Trento|
+-------------+-----------------------------------------+----------------------------------------+

In [0]:
dati_covid_regioni = dati_covid_regioni.replace('P.A. Trento', value='Trento', subset='denominazione_regione')
dati_covid_regioni = dati_covid_regioni.replace('P.A. Bolzano', value='Bolzano', subset='denominazione_regione')

Si organizzano i dati in più dataframe.

In [0]:
colonne_base = ['data', 'codice_regione', 'denominazione_regione', 'codice_nuts_1', 'codice_nuts_2']
colonne_casi = ['totale_positivi', 'variazione_totale_positivi', 'nuovi_positivi', 'isolamento_domiciliare', 
                'dimessi_guariti', 'deceduti', 'totale_casi', 'tamponi']
colonne_ospedali = ['ricoverati_con_sintomi', 'terapia_intensiva', 'totale_ospedalizzati']

dati_covid_regioni_casi = dati_covid_regioni.select(colonne_base + colonne_casi)
dati_covid_regioni_ospedali = dati_covid_regioni.select(colonne_base + colonne_ospedali)

In [0]:
dati_covid_regioni_ospedali.show()

+-------------------+--------------+---------------------+-------------+-------------+----------------------+-----------------+--------------------+
 data|codice_regione|denominazione_regione|codice_nuts_1|codice_nuts_2|ricoverati_con_sintomi|terapia_intensiva|totale_ospedalizzati|
+-------------------+--------------+---------------------+-------------+-------------+----------------------+-----------------+--------------------+
2020-02-24 18:00:00| 13| Abruzzo| ITF| ITF1| 0| 0| 0|
2020-02-24 18:00:00| 17| Basilicata| ITF| ITF5| 0| 0| 0|
2020-02-24 18:00:00| 18| Calabria| ITF| ITF6| 0| 0| 0|
2020-02-24 18:00:00| 15| Campania| ITF| ITF3| 0| 0| 0|
2020-02-24 18:00:00| 8| Emilia-Romagna| ITH| ITH5| 10| 2| 12|
2020-02-24 18:00:00| 6| Friuli Venezia Gi...| ITH| ITH4| 0| 0| 0|
2020-02-24 18:00:00| 12| Lazio| ITI| ITI4| 1| 1| 2|
2020-02-24 18:00:00| 7| Liguria| ITC| ITC3| 0| 0| 0|
2020-02-24 18:00:00| 3| Lombardia| ITC| ITC4| 76| 19| 95|
2020-02-24 18:00:00| 11| Marche| ITI| ITI3| 0| 0| 0|
2020-02-24 18:00:00| 14| Molise| ITF| ITF2| 0| 0| 0|
2020-02-24 18:00:00| 21| Bolzano| 500340| ITH| 0| 0| 0|
2020-02-24 18:00:00| 21| Bolzano| ITH| ITH1| 0| 0| 0|
2020-02-24 18:00:00| 22| Trento| ITH| ITH2| 0| 0| 0|
2020-02-24 18:00:00| 1| Piemonte| ITC| ITC1| 2| 0| 2|
2020-02-24 18:00:00| 16| Puglia| ITF| ITF4| 0| 0| 0|
2020-02-24 18:00:00| 20| Sardegna| ITG| ITG2| 0| 0| 0|
2020-02-24 18:00:00| 19| Sicilia| ITG| ITG1| 0| 0| 0|
2020-02-24 18:00:00| 9| Toscana| ITI| ITI1| 0| 0| 0|
2020-02-24 18:00:00| 10| Umbria| 172410| ITI| 0| 0| 0|
+-------------------+--------------+---------------------+-------------+-------------+----------------------+-----------------+--------------------+
only showing top 20 rows